In [180]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import kagglehub
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import models
from PIL import Image

In [181]:
path = kagglehub.dataset_download("atulyakumar98/pothole-detection-dataset")

normal_dir = os.path.join(path, 'normal')
potholes_dir = os.path.join(path, 'potholes')
print("Normal images path:", normal_dir)
print("Potholes images path:", potholes_dir)

Normal images path: /Users/ybr/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4/normal
Potholes images path: /Users/ybr/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4/potholes


In [182]:
print("Dataset downloaded to:", path)

Dataset downloaded to: /Users/ybr/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4


In [183]:
class PotholeDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_names = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(image_path).convert("RGB")

        label = 0 if 'normal' in self.image_dir else 1

        if self.transform:
            image = self.transform(image)

        return image, label

In [184]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


normal_dataset = PotholeDataset(image_dir=normal_dir, transform=transform)
potholes_dataset = PotholeDataset(image_dir=potholes_dir, transform=transform)

combined_dataset = normal_dataset + potholes_dataset

train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(combined_dataset, batch_size=32, shuffle=False)

In [185]:
# MobileNetV2 Model for Classification
mobilenet_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenet_model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(mobilenet_model.last_channel, 2),
)

In [186]:
# InceptionV3 Model for Classification
inception_model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
inception_model.fc = nn.Linear(inception_model.fc.in_features, 2)

In [187]:
#ResNet50 Model for Classification
resnet_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2) # Match the number of classes

In [188]:
#ResNet101 Model for Classification
resnet_101_model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
resnet_101_model.fc = nn.Linear(resnet_101_model.fc.in_features, 2) # Match the number of classes

In [189]:
# VGG19 Model for Classification
vgg19_model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
vgg19_model.classifier[6] = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(vgg19_model.classifier[6].in_features, 2),
)

In [190]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilenet_model = mobilenet_model.to(device)
resnet_model = resnet_model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
mobilenet_optimizer = optim.Adam(mobilenet_model.classifier.parameters(), lr=0.001)
vgg19_optimizer = optim.Adam(vgg19_model.classifier.parameters(), lr=0.001)
resnet_optimizer = optim.Adam(resnet_model.fc.parameters(), lr=0.001)
resnet_101_optimizer = optim.Adam(resnet_101_model.fc.parameters(), lr=0.001)
inception_optimizer = optim.Adam(inception_model.fc.parameters(), lr=0.001)

In [192]:
def train_classification_model(model, optimizer, train_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        runningLoss = 0.0
        correctPredictions = 0.0
        totalPredictions = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            runningLoss += loss.item()
            _, predicted = torch.max(outputs, 1)
            totalPredictions += labels.size(0)
            correctPredictions += (predicted == labels).sum().item()

        epochLoss = runningLoss / len(train_loader)
        epochAccuracy = (correctPredictions / totalPredictions) * 100
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epochLoss:.4f}, Accuracy: {epochAccuracy:.2f}%")

In [193]:
def evaluate_classification_model(model, test_loader):
    model.eval()
    correctPredictions = 0.0
    totalPredictions = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            totalPredictions += labels.size(0)
            correctPredictions += (predicted == labels).sum().item()

    accuracy = (correctPredictions / totalPredictions) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

In [ ]:
print("Training ResNet101...")
train_classification_model(resnet_101_model, resnet_101_optimizer, train_loader)
evaluate_classification_model(resnet_101_model, test_loader)

Training ResNet101...
Epoch 1/10, Loss: 0.7177, Accuracy: 50.07%
Epoch 2/10, Loss: 0.7141, Accuracy: 51.69%
Epoch 3/10, Loss: 0.7172, Accuracy: 51.40%
Epoch 4/10, Loss: 0.7193, Accuracy: 49.05%
Epoch 5/10, Loss: 0.7173, Accuracy: 49.34%
Epoch 6/10, Loss: 0.7210, Accuracy: 50.22%
Epoch 7/10, Loss: 0.7151, Accuracy: 49.93%
Epoch 8/10, Loss: 0.7181, Accuracy: 50.51%
Epoch 9/10, Loss: 0.7107, Accuracy: 49.93%
Epoch 10/10, Loss: 0.7188, Accuracy: 49.34%


In [ ]:
print("Training VGG19...")
train_classification_model(vgg19_model, vgg19_optimizer, train_loader)
evaluate_classification_model(vgg19_model, test_loader)

Training VGG19...
Epoch 1/10, Loss: 0.2425, Accuracy: 98.68%
Epoch 2/10, Loss: 0.0937, Accuracy: 99.56%
Epoch 3/10, Loss: 0.3326, Accuracy: 98.53%


KeyboardInterrupt: 

In [ ]:
# Choose model and train
print("Training MobileNetV2...")
train_classification_model(mobilenet_model, mobilenet_optimizer, train_loader)
evaluate_classification_model(mobilenet_model, test_loader)







In [ ]:
print("Training ResNet50...")
train_classification_model(resnet_model, resnet_optimizer, train_loader)
evaluate_classification_model(resnet_model, test_loader)
